In [1]:
import pandas as pd
import geopy
import osmnx as ox
import networkx as nx
from geopy import distance

PROJ: proj_create_from_database: Cannot find proj.db


In [2]:
df = pd.read_csv('data/Dementia_ISRID2_Hanna_2017_Clean_For_R_v2_Dec8_edited.csv', encoding = "ISO-8859-1", index_col=0)

In [3]:
df.columns

Index(['Data Source', 'Country', 'State/Province/Region', 'Incident Type',
       'Incident Date Time', 'Incident Year', 'Incident Month', 'Incident Day',
       'Time', 'City', 'County', 'EcoRegion Domain', 'Population Density',
       'Terrain', 'Subject Category', 'Subject Sub-Category',
       'Subject Activity', 'Scenario', 'Group Type', '# Lost', 'Age', 'Sex',
       'Weight (Kg)', 'Height (Cm)', 'Physical Fitness', 'Mental Fitness',
       'Experience', 'Clothing', 'Personality', 'Subject Status',
       'Notify hours', 'Search hours', 'Total Time Lost', 'TTL Hours',
       'IPP Type', 'IPP Classification', 'IPP Coord. ', 'IPPAccuracy (remove)',
       'Destination Coord. (remove)', 'Temp/H', 'Temp L',
       'Wind (kph) (remove)', 'Weather', 'Snow (remove)', 'Rain (remove)',
       'Light (remove)', 'Investigative Find  (remove)',
       'Inv Find Details (remove)', 'Suspended (remove)',
       'Subject Found feature', 'Found Secondary (remove)',
       'Mobility Responsiveness

In [97]:
df['Comments'].values

array([nan,
       '71 year old male suffering Alzheimers, canoeing in bay.  Deceased; found by helicopter.',
       'Left at 0630 to drive 14km to Jondaryan.  Drove through. Ran out of fuel 260 km later, still believed he was heading for Jondaryan. Located 4km E. of Wallumbilla.',
       ...,
       'Lost party arrived in area with two others with the intention of walking up the \n\ntrack to the Kaite Falls. Due to the steepness of the track the other two in \n\nthe party returned to carpark. Lost party continues.\n\nLost party following track. Female losses balance and falls down bank dragging \n\nhusband with her. Not able to climb back up onto track due to steepness.\n\nLost party followed river for a short period. Realised that they were lost and \n\nremained in that position until located.\n\n\n\nSearch method: Tracks all check prior to the arrival of SAR squad by Police and \n\nlocals. Not located. Due to age and medical concerns of lost party urgent \n\nresponse required. Helic

In [44]:
df[['IPP Coord. ','Find Coord']].dropna()

,IPP Coord.,Find Coord
Incident #,,
5,Map 16 F3,Map 16 G5
6,8125 04370 59757,8125 04365 59742
18,205 H10,205 J11
19,GR871211,GR821239
21,23'48.8S 151'17.2E,23'49S 151'17.2E
...,...,...
3253,"42.17615, -74.03016","42.19392, -74.02443"
3254,"44.11271, -73.92375","44.12572, -73.9779"
3255,"42.44672, -78.89963","42.44672, -78.89963"


In [49]:
sample = df.dropna(subset=['IPP Coord. '])[df.dropna(subset=['IPP Coord. '])['IPP Coord. '].str.contains('-')][['Distance IPP (km)','IPP Coord. ','Find Coord','Country','County','State/Province/Region']].dropna()

In [50]:
sample['latitude_IPP'] = sample['IPP Coord. '].str.split('-').str[0].str.replace(',','')
sample['latitude_IPP'] = sample['latitude_IPP'].astype(float)
sample['longitude_IPP'] = '-' + sample['IPP Coord. '].str.split('-').str[1].str.replace(',','')
sample['longitude_IPP'] = sample['longitude_IPP'].astype(float)
sample['latitude_find'] = sample['Find Coord'].str.split('-').str[0].str.replace(',','')
sample['latitude_find'] = sample['latitude_find'].astype(float)
sample['longitude_find'] = '-' + sample['Find Coord'].str.split('-').str[1].str.replace(',','')
sample['longitude_find'] = sample['longitude_find'].astype(float)
sample['IPP_Coords'] = list(zip(sample.latitude_IPP, sample.longitude_IPP)) 
sample['Find_Coords'] = list(zip(sample.latitude_find, sample.longitude_find)) 

In [51]:
sample

,Distance IPP (km),IPP Coord.,Find Coord,Country,County,State/Province/Region,latitude_IPP,longitude_IPP,latitude_find,longitude_find,IPP_Coords,Find_Coords
Incident #,,,,,,,,,,,,
336,0.402493,34.65-112.5,34.6518666585286-112.496233336131,US,YAVAPAI,AZ,34.650000,-112.500000,34.651867,-112.496233,"(34.65, -112.5)","(34.6518666585286, -112.496233336131)"
337,3.282059,31.3799999872843-109.885833358765,31.3721000035604-109.919099998474,US,COCHISE,AZ,31.380000,-109.885833,31.372100,-109.919100,"(31.3799999872843, -109.885833358765)","(31.3721000035604, -109.919099998474)"
340,1.960362,33.5823500315348-111.215133333206,33.5966999689738-111.202783330282,US,MARICOPA,AZ,33.582350,-111.215133,33.596700,-111.202783,"(33.5823500315348, -111.215133333206)","(33.5966999689738, -111.202783330282)"
341,0.549781,32.3236666679382-111.073333334923,32.3187166531881-111.072949997584,US,PIMA,AZ,32.323667,-111.073333,32.318717,-111.072950,"(32.3236666679382, -111.073333334923)","(32.3187166531881, -111.072949997584)"
342,6.400926,35.2655833403269-114.026699999968,35.2994166692098-114.083666666349,US,MOHAVE,AZ,35.265583,-114.026700,35.299417,-114.083667,"(35.2655833403269, -114.026699999968)","(35.2994166692098, -114.083666666349)"
...,...,...,...,...,...,...,...,...,...,...,...,...
3244,0.986931,"34.8311014, -85.3312615","34.8284681, -85.3209278",US,Walker,GA,34.831101,-85.331261,34.828468,-85.320928,"(34.8311014, -85.3312615)","(34.8284681, -85.3209278)"
3249,0.000000,"36.7225, -84.1682166666667","36.7225, -84.1682166666667",US,Whitley,KY,36.722500,-84.168217,36.722500,-84.168217,"(36.7225, -84.1682166666667)","(36.7225, -84.1682166666667)"
3253,2.030181,"42.17615, -74.03016","42.19392, -74.02443",US,Greene,NY,42.176150,-74.030160,42.193920,-74.024430,"(42.17615, -74.03016)","(42.19392, -74.02443)"


In [52]:
distance.distance(sample['IPP_Coords'].iloc[0], sample['Find_Coords'].iloc[0]).km

0.40263705411491607

In [53]:
sample['distance'] = sample.apply(lambda row: distance.distance(row['IPP_Coords'],row['Find_Coords']).km, axis=1)

In [54]:
sample

,Distance IPP (km),IPP Coord.,Find Coord,Country,County,State/Province/Region,latitude_IPP,longitude_IPP,latitude_find,longitude_find,IPP_Coords,Find_Coords,distance
Incident #,,,,,,,,,,,,,
336,0.402493,34.65-112.5,34.6518666585286-112.496233336131,US,YAVAPAI,AZ,34.650000,-112.500000,34.651867,-112.496233,"(34.65, -112.5)","(34.6518666585286, -112.496233336131)",0.402637
337,3.282059,31.3799999872843-109.885833358765,31.3721000035604-109.919099998474,US,COCHISE,AZ,31.380000,-109.885833,31.372100,-109.919100,"(31.3799999872843, -109.885833358765)","(31.3721000035604, -109.919099998474)",3.283555
340,1.960362,33.5823500315348-111.215133333206,33.5966999689738-111.202783330282,US,MARICOPA,AZ,33.582350,-111.215133,33.596700,-111.202783,"(33.5823500315348, -111.215133333206)","(33.5966999689738, -111.202783330282)",1.961510
341,0.549781,32.3236666679382-111.073333334923,32.3187166531881-111.072949997584,US,PIMA,AZ,32.323667,-111.073333,32.318717,-111.072950,"(32.3236666679382, -111.073333334923)","(32.3187166531881, -111.072949997584)",0.550105
342,6.400926,35.2655833403269-114.026699999968,35.2994166692098-114.083666666349,US,MOHAVE,AZ,35.265583,-114.026700,35.299417,-114.083667,"(35.2655833403269, -114.026699999968)","(35.2994166692098, -114.083666666349)",6.399042
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3244,0.986931,"34.8311014, -85.3312615","34.8284681, -85.3209278",US,Walker,GA,34.831101,-85.331261,34.828468,-85.320928,"(34.8311014, -85.3312615)","(34.8284681, -85.3209278)",0.989405
3249,0.000000,"36.7225, -84.1682166666667","36.7225, -84.1682166666667",US,Whitley,KY,36.722500,-84.168217,36.722500,-84.168217,"(36.7225, -84.1682166666667)","(36.7225, -84.1682166666667)",0.000000
3253,2.030181,"42.17615, -74.03016","42.19392, -74.02443",US,Greene,NY,42.176150,-74.030160,42.193920,-74.024430,"(42.17615, -74.03016)","(42.19392, -74.02443)",2.029802


In [55]:
ox.config(use_cache=True, log_console=True)

In [72]:
graph = ox.graph.graph_from_point(center_point = sample.iloc[4]['IPP_Coords'], dist=1000,network_type='all')

In [57]:
sample.iloc[0]['IPP_Coords']

(34.65, -112.5)

In [73]:
optimizer = 'time'  # or length
# find the nearest node to the start location
orig_node = ox.distance.nearest_nodes(graph, sample.iloc[4]['IPP_Coords'])
# find the nearest node to the end location
dest_node = ox.distance.nearest_nodes(graph, sample.iloc[4]['Find_Coords'])
#  find the shortest path
shortest_route = nx.shortest_path(graph,
                                  orig_node,
                                  dest_node,
                                  weight=optimizer)

C:\ProgramData\Anaconda3\envs\agewell\lib\site-packages\osmnx\distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)
C:\ProgramData\Anaconda3\envs\agewell\lib\site-packages\osmnx\distance.py:356: UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


In [74]:
shortest_route

[175182145, 175171325, 174682772, 174682782, 372578798, 175148827]

In [75]:
shortest_route_map = ox.plot_route_folium(graph, shortest_route)
shortest_route_map

In [78]:
nx.shortest_path_length(graph, orig_node, dest_node, weight='length') / 1000

1.2058659999999999

In [91]:
def routing(row,optimizer='time',network_type='walk'):
    """Find shortest paths between IPP and Find Coordinates and output map and distance"""
    graph = ox.graph.graph_from_point(center_point=row['IPP_Coords'], dist=1000, network_type=network_type)
    # find the nearest node to the start location
    orig_node = ox.distance.nearest_nodes(graph, X=row['longitude_IPP'],Y=row['latitude_IPP'])
    # find the nearest node to the end location
    dest_node = ox.distance.nearest_nodes(graph, X=row['longitude_find'],Y=row['latitude_find'])
    #  find the shortest path
    shortest_route = nx.shortest_path(graph,
                                      orig_node,
                                      dest_node,
                                      weight=optimizer)
    shortest_route_map = ox.plot_route_folium(graph, shortest_route)
    dist = nx.shortest_path_length(graph, orig_node, dest_node, weight='length') / 1000
    dist_actual = row['distance']
    print(f'Shortest distance is {dist}km')
    print(f'Distance between IPP and Find Coordinates is {dist_actual}km')
    return shortest_route_map

In [94]:
routing(sample.iloc[2],optimizer='time',network_type='walk')

Shortest distance is 1.4439879999999998km
Distance between IPP and Find Coordinates is 1.9615102863663985km
